### Importa pacotes

In [31]:
import pandas as pd
import mysql.connector
from mysql.connector import Error

### Define funções

In [3]:
#Função de nome dos meses
def func_mes(row):
    if row['MES_EXT'] == 'Jan':
        return '01'
    elif row['MES_EXT'] =='Fev':
        return '02'
    elif row['MES_EXT'] =='Mar':
        return '03'
    elif row['MES_EXT'] =='Abr':
        return '04'
    elif row['MES_EXT'] =='Mai':
        return '05'
    elif row['MES_EXT'] =='Jun':
        return '06'
    elif row['MES_EXT'] =='Jul':
        return '07'
    elif row['MES_EXT'] =='Ago':
        return '08'
    elif row['MES_EXT'] =='Set':
        return '09'
    elif row['MES_EXT'] =='Out':
        return '10'
    elif row['MES_EXT'] =='Nov':
        return '11'
    elif row['MES_EXT'] =='Dez':
        return '12'
    else:
        return row['MES_EXT']

#Função de códigos de UF
def func_uf(row):
    if row['ESTADO_TRIM'] == 'RONDÔNIA':
        return 'RO'
    elif row['ESTADO_TRIM'] =='ACRE':
        return 'AC'
    elif row['ESTADO_TRIM'] =='AMAZONAS':
        return 'AM'
    elif row['ESTADO_TRIM'] =='RORAIMA':
        return 'RR'
    elif row['ESTADO_TRIM'] =='PARÁ':
        return 'PA'
    elif row['ESTADO_TRIM'] =='AMAPÁ':
        return 'AP'
    elif row['ESTADO_TRIM'] =='TOCANTINS':
        return 'TO'
    elif row['ESTADO_TRIM'] =='MARANHÃO':
        return 'MA'
    elif row['ESTADO_TRIM'] =='PIAUÍ':
        return 'PI'
    elif row['ESTADO_TRIM'] =='CEARÁ':
        return 'CE'
    elif row['ESTADO_TRIM'] =='RIO GRANDE DO NORTE':
        return 'RN'
    elif row['ESTADO_TRIM'] =='PARAÍBA':
        return 'PB'
    elif row['ESTADO_TRIM'] =='PERNAMBUCO':
        return 'PE'
    elif row['ESTADO_TRIM'] =='ALAGOAS':
        return 'AL'
    elif row['ESTADO_TRIM'] =='SERGIPE':
        return 'SE'
    elif row['ESTADO_TRIM'] =='BAHIA':
        return 'BA'
    elif row['ESTADO_TRIM'] =='MINAS GERAIS':
        return 'MG'
    elif row['ESTADO_TRIM'] =='ESPÍRITO SANTO':
        return 'ES'
    elif row['ESTADO_TRIM'] =='RIO DE JANEIRO':
        return 'RJ'
    elif row['ESTADO_TRIM'] =='SÃO PAULO':
        return 'SP'
    elif row['ESTADO_TRIM'] =='PARANÁ':
        return 'PR'
    elif row['ESTADO_TRIM'] =='SANTA CATARINA':
        return 'SC'
    elif row['ESTADO_TRIM'] =='RIO GRANDE DO SUL':
        return 'RS'
    elif row['ESTADO_TRIM'] =='MATO GROSSO DO SUL':
        return 'MS'
    elif row['ESTADO_TRIM'] =='MATO GROSSO':
        return 'MT'
    elif row['ESTADO_TRIM'] =='GOIÁS':
        return 'GO'
    elif row['ESTADO_TRIM'] =='DISTRITO FEDERAL':
        return 'DF'
    else:
        return row['ESTADO_TRIM']

### Importa dados

#### Sales of oil derivative fuels by UF and product

In [4]:
#Lendo excel
df_Oil_UF_prod = pd.read_excel (r'C:\Users\cysb\OneDrive\Documentos\Python Scripts\Raizen\data\vendas-combustiveis-m3-Libre.xls', sheet_name = 'DPCache_m3', usecols='A:Q', nrows=4537)


#### Sales of diesel by UF and type

In [5]:
#Lendo excel
df_Oil_UF_type = pd.read_excel (r'C:\Users\cysb\OneDrive\Documentos\Python Scripts\Raizen\data\vendas-combustiveis-m3-Libre.xls', sheet_name = 'DPCache_m3_2', usecols='A:Q', nrows=1081)


### Despivoteia dados

#### Sales of oil derivative fuels by UF and product

In [6]:
#Colunas viram linhas
df_Oil_UF_prod_unpivot = df_Oil_UF_prod.melt(id_vars = ['COMBUSTÍVEL','ANO','REGIÃO','ESTADO'], var_name = 'MES_EXT', value_name = 'QTDE')

In [7]:
df_Oil_UF_prod_unpivot

,COMBUSTÍVEL,ANO,REGIÃO,ESTADO,MES_EXT,QTDE
0,GASOLINA C (m3),2000,REGIÃO NORTE,RONDÔNIA,Jan,136073.253000
1,GASOLINA C (m3),2000,REGIÃO NORTE,ACRE,Jan,3358.346000
2,GASOLINA C (m3),2000,REGIÃO NORTE,AMAZONAS,Jan,20766.918000
3,GASOLINA C (m3),2000,REGIÃO NORTE,RORAIMA,Jan,3716.032000
4,GASOLINA C (m3),2000,REGIÃO NORTE,PARÁ,Jan,29755.907000
...,...,...,...,...,...,...
58963,GLP (m3),2020,REGIÃO SUL,RIO GRANDE DO SUL,TOTAL,66749.367754
58964,GLP (m3),2020,REGIÃO CENTRO-OESTE,MATO GROSSO DO SUL,TOTAL,15037.528986
58965,GLP (m3),2020,REGIÃO CENTRO-OESTE,MATO GROSSO,TOTAL,20677.483696
58966,GLP (m3),2020,REGIÃO CENTRO-OESTE,GOIÁS,TOTAL,46581.878623


#### Sales of diesel by UF and type

In [8]:
#Colunas viram linhas
df_Oil_UF_type_unpivot = df_Oil_UF_type.melt(id_vars = ['COMBUSTÍVEL','ANO','REGIÃO','ESTADO'], var_name = 'MES_EXT', value_name = 'QTDE')


In [9]:
df_Oil_UF_type_unpivot

,COMBUSTÍVEL,ANO,REGIÃO,ESTADO,MES_EXT,QTDE
0,ÓLEO DIESEL S-10 (m3),2013,REGIÃO NORTE,RONDÔNIA,Jan,81453.67
1,ÓLEO DIESEL S-10 (m3),2013,REGIÃO NORTE,ACRE,Jan,1483.00
2,ÓLEO DIESEL S-10 (m3),2013,REGIÃO NORTE,AMAZONAS,Jan,6836.30
3,ÓLEO DIESEL S-10 (m3),2013,REGIÃO NORTE,RORAIMA,Jan,1475.30
4,ÓLEO DIESEL S-10 (m3),2013,REGIÃO NORTE,PARÁ,Jan,40913.48
...,...,...,...,...,...,...
14035,ÓLEO DIESEL (OUTROS ) (m3),2020,REGIÃO SUL,RIO GRANDE DO SUL,TOTAL,10.00
14036,ÓLEO DIESEL (OUTROS ) (m3),2020,REGIÃO CENTRO-OESTE,MATO GROSSO DO SUL,TOTAL,0.00
14037,ÓLEO DIESEL (OUTROS ) (m3),2020,REGIÃO CENTRO-OESTE,MATO GROSSO,TOTAL,45.00
14038,ÓLEO DIESEL (OUTROS ) (m3),2020,REGIÃO CENTRO-OESTE,GOIÁS,TOTAL,41.40


### Campos auxiliares

#### Sales of oil derivative fuels by UF and product

In [12]:
#Limpa linhas de totais
df_Oil_UF_prod_unpivot = df_Oil_UF_prod_unpivot[(df_Oil_UF_prod_unpivot['MES_EXT'] != 'TOTAL')]

#Limpa espaços
df_Oil_UF_prod_unpivot['ESTADO_TRIM'] = df_Oil_UF_prod_unpivot['ESTADO'].str.strip()

#Aplicando nome dos meses
df_Oil_UF_prod_unpivot['NR_MES'] = df_Oil_UF_prod_unpivot.apply(func_mes, axis=1)


#### Sales of diesel by UF and type

In [15]:
#Limpa linhas de totais
df_Oil_UF_type_unpivot = df_Oil_UF_type_unpivot[(df_Oil_UF_type_unpivot['MES_EXT'] != 'TOTAL')]

#Limpa espaços
df_Oil_UF_type_unpivot['ESTADO_TRIM'] = df_Oil_UF_type_unpivot['ESTADO'].str.strip()

#Aplicando nome dos meses
df_Oil_UF_type_unpivot['NR_MES'] = df_Oil_UF_type_unpivot.apply(func_mes, axis=1)


In [16]:
df_Oil_UF_type_unpivot

,COMBUSTÍVEL,ANO,REGIÃO,ESTADO,MES_EXT,QTDE,ESTADO_TRIM,NR_MES
0,ÓLEO DIESEL S-10 (m3),2013,REGIÃO NORTE,RONDÔNIA,Jan,81453.67,RONDÔNIA,01
1,ÓLEO DIESEL S-10 (m3),2013,REGIÃO NORTE,ACRE,Jan,1483.00,ACRE,01
2,ÓLEO DIESEL S-10 (m3),2013,REGIÃO NORTE,AMAZONAS,Jan,6836.30,AMAZONAS,01
3,ÓLEO DIESEL S-10 (m3),2013,REGIÃO NORTE,RORAIMA,Jan,1475.30,RORAIMA,01
4,ÓLEO DIESEL S-10 (m3),2013,REGIÃO NORTE,PARÁ,Jan,40913.48,PARÁ,01
...,...,...,...,...,...,...,...,...
12955,ÓLEO DIESEL (OUTROS ) (m3),2020,REGIÃO SUL,RIO GRANDE DO SUL,Dez,10.00,RIO GRANDE DO SUL,12
12956,ÓLEO DIESEL (OUTROS ) (m3),2020,REGIÃO CENTRO-OESTE,MATO GROSSO DO SUL,Dez,0.00,MATO GROSSO DO SUL,12
12957,ÓLEO DIESEL (OUTROS ) (m3),2020,REGIÃO CENTRO-OESTE,MATO GROSSO,Dez,90.00,MATO GROSSO,12
12958,ÓLEO DIESEL (OUTROS ) (m3),2020,REGIÃO CENTRO-OESTE,GOIÁS,Dez,NaN,GOIÁS,12


### Campos finais

#### Sales of oil derivative fuels by UF and product

In [17]:
#Concatenando e convertendo para data
df_Oil_UF_prod_unpivot['year_month'] = pd.to_datetime(df_Oil_UF_prod_unpivot["ANO"].astype(str) + df_Oil_UF_prod_unpivot["NR_MES"] + '01', format='%Y%m%d', errors='coerce')

#Aplicando códigos de UF
df_Oil_UF_prod_unpivot['uf'] = df_Oil_UF_prod_unpivot.apply(func_uf, axis=1)

#Formatando campos
df_Oil_UF_prod_unpivot['product'] = df_Oil_UF_prod_unpivot['COMBUSTÍVEL'].apply(lambda st: st[:st.find("(")])

df_Oil_UF_prod_unpivot['unit'] = df_Oil_UF_prod_unpivot['COMBUSTÍVEL'].apply(lambda st: st[st.find("(")+1:st.find(")")])

df_Oil_UF_prod_unpivot['volume'] = df_Oil_UF_prod_unpivot['QTDE']

df_Oil_UF_prod_unpivot['created_at'] = pd.to_datetime("today")

#### Sales of diesel by UF and type

In [19]:
#Concatenando e convertendo para data
df_Oil_UF_type_unpivot['year_month'] = pd.to_datetime(df_Oil_UF_type_unpivot["ANO"].astype(str) + df_Oil_UF_type_unpivot["NR_MES"] + '01', format='%Y%m%d', errors='coerce')

#Aplicando códigos de UF
df_Oil_UF_type_unpivot['uf'] = df_Oil_UF_type_unpivot.apply(func_uf, axis=1)

#Formatando campos
df_Oil_UF_type_unpivot['product'] = df_Oil_UF_type_unpivot['COMBUSTÍVEL'].apply(lambda st: st[:st.find("(")])

df_Oil_UF_type_unpivot['unit'] = df_Oil_UF_type_unpivot['COMBUSTÍVEL'].apply(lambda st: st[st.find("(")+1:st.find(")")])

df_Oil_UF_type_unpivot['volume'] = df_Oil_UF_type_unpivot['QTDE']

df_Oil_UF_type_unpivot['created_at'] = pd.to_datetime("today")

In [20]:
df_Oil_UF_type_unpivot

,COMBUSTÍVEL,ANO,REGIÃO,ESTADO,MES_EXT,QTDE,ESTADO_TRIM,NR_MES,year_month,uf,product,unit,volume,created_at
0,ÓLEO DIESEL S-10 (m3),2013,REGIÃO NORTE,RONDÔNIA,Jan,81453.67,RONDÔNIA,01,2013-01-01,RO,ÓLEO DIESEL S-10,m3,81453.67,2022-07-17 19:26:47.658185
1,ÓLEO DIESEL S-10 (m3),2013,REGIÃO NORTE,ACRE,Jan,1483.00,ACRE,01,2013-01-01,AC,ÓLEO DIESEL S-10,m3,1483.00,2022-07-17 19:26:47.658185
2,ÓLEO DIESEL S-10 (m3),2013,REGIÃO NORTE,AMAZONAS,Jan,6836.30,AMAZONAS,01,2013-01-01,AM,ÓLEO DIESEL S-10,m3,6836.30,2022-07-17 19:26:47.658185
3,ÓLEO DIESEL S-10 (m3),2013,REGIÃO NORTE,RORAIMA,Jan,1475.30,RORAIMA,01,2013-01-01,RR,ÓLEO DIESEL S-10,m3,1475.30,2022-07-17 19:26:47.658185
4,ÓLEO DIESEL S-10 (m3),2013,REGIÃO NORTE,PARÁ,Jan,40913.48,PARÁ,01,2013-01-01,PA,ÓLEO DIESEL S-10,m3,40913.48,2022-07-17 19:26:47.658185
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12955,ÓLEO DIESEL (OUTROS ) (m3),2020,REGIÃO SUL,RIO GRANDE DO SUL,Dez,10.00,RIO GRANDE DO SUL,12,2020-12-01,RS,ÓLEO DIESEL,OUTROS,10.00,2022-07-17 19:26:47.658185
12956,ÓLEO DIESEL (OUTROS ) (m3),2020,REGIÃO CENTRO-OESTE,MATO GROSSO DO SUL,Dez,0.00,MATO GROSSO DO SUL,12,2020-12-01,MS,ÓLEO DIESEL,OUTROS,0.00,2022-07-17 19:26:47.658185
12957,ÓLEO DIESEL (OUTROS ) (m3),2020,REGIÃO CENTRO-OESTE,MATO GROSSO,Dez,90.00,MATO GROSSO,12,2020-12-01,MT,ÓLEO DIESEL,OUTROS,90.00,2022-07-17 19:26:47.658185
12958,ÓLEO DIESEL (OUTROS ) (m3),2020,REGIÃO CENTRO-OESTE,GOIÁS,Dez,NaN,GOIÁS,12,2020-12-01,GO,ÓLEO DIESEL,OUTROS,NaN,2022-07-17 19:26:47.658185


### Testa totais

#### Sales of oil derivative fuels by UF and product

In [21]:
#Teste de totais
df_Oil_UF_prod_unpivot[(df_Oil_UF_prod_unpivot['ANO'] == 2020)].groupby(['MES_EXT','NR_MES','ANO']).sum().sort_values(by='NR_MES')

,,,QTDE,volume
MES_EXT,NR_MES,ANO,,
Jan,01,2020,1.513507e+07,1.513507e+07
Fev,02,2020,1.230695e+07,1.230695e+07
Mar,03,2020,1.485362e+07,1.485362e+07
Abr,04,2020,1.639673e+07,1.639673e+07
Mai,05,2020,1.328373e+07,1.328373e+07
Jun,06,2020,1.121496e+07,1.121496e+07
Jul,07,2020,2.032894e+07,2.032894e+07
Ago,08,2020,1.507544e+07,1.507544e+07
Set,09,2020,1.070232e+07,1.070232e+07


#### Sales of diesel by UF and type

In [22]:
#Teste de totais
df_Oil_UF_type_unpivot[(df_Oil_UF_type_unpivot['ANO'] == 2013)].groupby(['MES_EXT','NR_MES','ANO']).sum().sort_values(by='NR_MES')

,,,QTDE,volume
MES_EXT,NR_MES,ANO,,
Jan,01,2013,8.021002e+06,8.021002e+06
Fev,02,2013,7.818515e+06,7.818515e+06
Mar,03,2013,6.937346e+06,6.937346e+06
Abr,04,2013,8.141449e+06,8.141449e+06
Mai,05,2013,9.221813e+06,9.221813e+06
Jun,06,2013,1.068028e+07,1.068028e+07
Jul,07,2013,9.732656e+06,9.732656e+06
Ago,08,2013,1.391075e+07,1.391075e+07
Set,09,2013,1.303868e+07,1.303868e+07


### Dataframe final

#### Sales of oil derivative fuels by UF and product

In [34]:
#Dataframe no modelo definido
df_Oil_By_UF_prod_final = df_Oil_UF_prod_unpivot[['year_month','uf','product','unit','volume','created_at']]

df_Oil_By_UF_prod_final = df_Oil_By_UF_prod_final.fillna(0)

df_Oil_By_UF_prod_final

,year_month,uf,product,unit,volume,created_at
0,2000-01-01,RO,GASOLINA C,m3,136073.253000,2022-07-17 19:26:36.832433
1,2000-01-01,AC,GASOLINA C,m3,3358.346000,2022-07-17 19:26:36.832433
2,2000-01-01,AM,GASOLINA C,m3,20766.918000,2022-07-17 19:26:36.832433
3,2000-01-01,RR,GASOLINA C,m3,3716.032000,2022-07-17 19:26:36.832433
4,2000-01-01,PA,GASOLINA C,m3,29755.907000,2022-07-17 19:26:36.832433
...,...,...,...,...,...,...
54427,2020-12-01,RS,GLP,m3,64045.161232,2022-07-17 19:26:36.832433
54428,2020-12-01,MS,GLP,m3,16281.139493,2022-07-17 19:26:36.832433
54429,2020-12-01,MT,GLP,m3,18321.987319,2022-07-17 19:26:36.832433
54430,2020-12-01,GO,GLP,m3,46850.585145,2022-07-17 19:26:36.832433


#### Sales of diesel by UF and type¶

In [33]:
#Dataframe no modelo definido
df_Oil_By_UF_type_final = df_Oil_UF_type_unpivot[['year_month','uf','product','unit','volume','created_at']]

df_Oil_By_UF_type_final = df_Oil_By_UF_type_final.fillna(0)

df_Oil_By_UF_type_final

,year_month,uf,product,unit,volume,created_at
0,2013-01-01,RO,ÓLEO DIESEL S-10,m3,81453.67,2022-07-17 19:26:47.658185
1,2013-01-01,AC,ÓLEO DIESEL S-10,m3,1483.00,2022-07-17 19:26:47.658185
2,2013-01-01,AM,ÓLEO DIESEL S-10,m3,6836.30,2022-07-17 19:26:47.658185
3,2013-01-01,RR,ÓLEO DIESEL S-10,m3,1475.30,2022-07-17 19:26:47.658185
4,2013-01-01,PA,ÓLEO DIESEL S-10,m3,40913.48,2022-07-17 19:26:47.658185
...,...,...,...,...,...,...
12955,2020-12-01,RS,ÓLEO DIESEL,OUTROS,10.00,2022-07-17 19:26:47.658185
12956,2020-12-01,MS,ÓLEO DIESEL,OUTROS,0.00,2022-07-17 19:26:47.658185
12957,2020-12-01,MT,ÓLEO DIESEL,OUTROS,90.00,2022-07-17 19:26:47.658185
12958,2020-12-01,GO,ÓLEO DIESEL,OUTROS,0.00,2022-07-17 19:26:47.658185


In [25]:
df_Oil_By_UF_type_final.dtypes

year_month    datetime64[ns]
uf                    object
product               object
unit                  object
volume               float64
created_at    datetime64[ns]
dtype: object

### Insere no banco

#### Monta ambiente BD MySQL

In [42]:
try:
    con = mysql.connector.connect(host='localhost', database='santexflix', user='UserCarga', password='UserCarga')
    if con.is_connected():
        cursor = con.cursor()
        print("You're connected to database")
        #Create tables
        cursor.execute("drop table if exists santexflix.tb_Oil_By_UF_type")
        print("Table tb_Oil_By_UF_type droped")
        cursor.execute("create table santexflix.tb_Oil_By_UF_type ( yearmonth datetime,uf varchar(2),product varchar(100),unit varchar(15),volume float,created_at datetime)")
        print("Table tb_Oil_By_UF_type created")
        cursor.execute("drop table if exists santexflix.tb_Oil_By_UF_prod")
        print("Table tb_Oil_By_UF_type droped")
        cursor.execute("create table santexflix.tb_Oil_By_UF_prod ( yearmonth datetime,uf varchar(2),product varchar(100),unit varchar(15),volume float,created_at datetime)")
        print("Table tb_Oil_By_UF_type created")
except Error as e:
            print("Error while connecting to MySQL", e)

You're connected to database
Table tb_Oil_By_UF_type droped
Table tb_Oil_By_UF_type created
Table tb_Oil_By_UF_type droped
Table tb_Oil_By_UF_type created


#### Sales of oil derivative fuels by UF and product

In [43]:
try:
    con = mysql.connector.connect(host='localhost', database='santexflix', user='UserCarga', password='UserCarga')
    if con.is_connected():
        cursor = con.cursor()
        print("You're connected to database")
        #Restart table
        cursor.execute("TRUNCATE TABLE santexflix.tb_Oil_By_UF_prod")
        print("Table restarted")
        #Loop through the data frame
        for i,row in df_Oil_By_UF_prod_final.iterrows():
            #here %S means string values 
            sql = "REPLACE INTO santexflix.tb_Oil_By_UF_prod (yearmonth ,uf ,product ,unit ,volume ,created_at) VALUES (%s,%s,%s,%s,%s,%s)"
            cursor.execute(sql, tuple(row))
            print("Record inserted")
            # the connection is not auto committed by default, so we must commit to save our changes
            con.commit()
        print("Load finish")
except Error as e:
            print("Error while connecting to MySQL", e)

You're connected to database
Table restarted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inse

Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record i

Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record i

Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record i

Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record i

...

Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record i

Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record i

Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record i

Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record i

Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record i

#### Sales of diesel by UF and type

In [44]:
try:
    con = mysql.connector.connect(host='localhost', database='santexflix', user='UserCarga', password='UserCarga')
    if con.is_connected():
        cursor = con.cursor()
        print("You're connected to database")
        #Restart table
        cursor.execute("TRUNCATE TABLE santexflix.tb_Oil_By_UF_type")
        print("Table restarted")
        #Loop through the data frame
        for i,row in df_Oil_By_UF_type_final.iterrows():
            #here %S means string values 
            sql = "REPLACE INTO santexflix.tb_Oil_By_UF_type (yearmonth ,uf ,product ,unit ,volume ,created_at) VALUES (%s,%s,%s,%s,%s,%s)"
            cursor.execute(sql, tuple(row))
            print("Record inserted")
            # the connection is not auto committed by default, so we must commit to save our changes
            con.commit()
        print("Load finish")
except Error as e:
            print("Error while connecting to MySQL", e)

You're connected to database
Table restarted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inse

Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record i

Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record i

Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record i

Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record i

...

Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record i

Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record i

Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record i

Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record i

Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Record inserted
Load finish
